# Bài toán cây xanh

*Mô hình hoá và mô phỏng bằng Python*

Bản quyền 2021 Allen Downey

Giấy phép: [Creative Commons Attribution-NonCommercial-ShareAlike 4.0 International](https://creativecommons.org/licenses/by-nc-sa/4.0/)

In [1]:
# cài đặt Pint nếu cần

try:
    import pint
except ImportError:
    !pip install pint

In [2]:
# tải về modsim.py nếu cần

from os.path import exists

filename = 'modsim.py'
if not exists(filename):
    from urllib.request import urlretrieve
    url = 'https://raw.githubusercontent.com/AllenDowney/ModSim/main/'
    local, _ = urlretrieve(url+filename, filename)
    print('Downloaded ' + local)

In [3]:
# nhập các hàm từ modsim

from modsim import *

## Mô hình hóa sự tăng trưởng của cây xanh

Nghiên cứu cụ thể này được dựa theo "[Height-Age Curves for Planted Stands of Douglas Fir, with Adjustments for Density](http://www.cfr.washington.edu/research.smc/working_papers/smc_working_paper_1.pdf)" (tạm dịch: các đường cong biểu hiện chiều cao-tuổi cây thông Douglas được gieo trồng, có điều chỉnh theo mật độ cây), một bài báo của nhóm tác giả Flewelling, Collier, Gonyea, Marshall, và Turnblom.

Bài báo cho ta các "đường cong chỉ số theo địa điểm", đây là những đường cong đồ thị biểu diên chiều cao ước tính của cây cao nhất trong một hàng cây thông Douglas như một hàm tuổi cây, đối với hàng trồng các cây cùng tuổi.

Tùy thuộc vào chất lượng của hàng ở từng địa điểm, cây có thể mọc nhanh hơn hoặc chậm hơn. Như vậy, mỗi đường đồ thị được nhận diện bởi một "chỉ số địa điểm" vốn phản ánh chất lượng của địa điểm này.

Tôi sẽ bắt đầu với vài số liệu của họ trong Bảng 1. Dưới đây là dãy các tuổi cây.

In [4]:
years = [2, 3, 4, 5, 6, 8, 10, 15, 20, 25, 30,
         35, 40, 45, 50, 55, 60, 65, 70]

Và đây là một dãy các chiều cao cây ở địa điểm có chỉ số 45, điều này cho thấy chiều cao ở tuổi cây 30 năm là 45 feet.

In [5]:
site45 = TimeSeries([1.4, 1.49, 1.75, 2.18, 2.78, 4.45, 6.74,
                    14.86, 25.39, 35.60, 45.00, 53.65, 61.60,
                    68.92, 75.66, 81.85, 87.56, 92.8, 97.63],
                    index=years)

Đây là dãy số liệu ở địa điểm có chỉ số 65.

In [6]:
site65 = TimeSeries([1.4, 1.56, 2.01, 2.76, 3.79, 6.64, 10.44, 
                    23.26, 37.65, 51.66, 65.00, 77.50, 89.07, 
                    99.66, 109.28, 117.96, 125.74, 132.68, 138.84],
                    index=years)

Và ở địa điểm có chỉ số 85.

In [7]:
site85 = TimeSeries([1.4, 1.8, 2.71, 4.09, 5.92, 10.73, 16.81, 
                 34.03, 51.26, 68.54, 85, 100.34, 114.33,
                 126.91, 138.06, 147.86, 156.39, 163.76, 170.10],
               index=years)

Các đường cong sẽ có hình dạng như dưới đây:

In [8]:
site85.plot(label='SI 85')
site65.plot(label='SI 65')
site45.plot(label='SI 45')
decorate(xlabel='Time (years)',
         ylabel='Height (feet)')

Tôi chỉ làm ví dụ với dữ liệu chỉ số (SI) là 65. Bạn có thể tập chạy lại sổ tính notebook này cho các đường cong khác.

In [9]:
data = site65

## Mô hình 1

Để bắt đầu, ta hãy giả sử rằng khả năng cây trồng tăng trọng thì bị hạn chế bởi diện tích nó phơi ra ngoài ánh mặt trời, còn tốc độ tăng trưởng (về khối lượng) thì tỉ lệ thuận với diện tích đó. Như vậy, ta có thể viết:

$$ m_{n+1} = m_n + \alpha A$$

trong đó $m_n$ là khối lượng cây tại bước thời gian $n$, $A$ là diện tích phơi nắng, còn $\alpha$ là một hệ số sinh trưởng mà ta chưa biết.

Để đi từ $m$ đến $A$, tôi sẽ giả sử thêm rằng khối lượng thì tỉ lệ với chiều cao lũy thừa một số mũ chưa biết khác:

$$ m = \beta h^D $$

trong đó $h$ là chiều cao, $eta$ là một hằng số tỉ lệ chưa biết, và $D$ là đại lượng liên hệ giữa chiều cao và khối lượng. Để bắt đầu, tôi sẽ giả sử $D=3$, và sẽ còn kiểm tra lại giả sử này sau.

Cuối cùng, tôi sẽ giả sử rằng diện tích thì tỉ lệ với bình phương của chiều cao:

$$ A = \gamma h^2$$

Tôi tính chiều cao theo feet, và chọn các đơn vị cho khối lượng và diện tích sao cho $\beta=1$ và $\gamma=1$.

Tổng hợp mọi thứ lại, ta có thể viết được một phương trình sai phân cho chiều cao:

$$ h_{n+1}^D = h_n^D + \alpha h_n^2 $$

Bây giờ ta hãy mô phỏng hệ thống này. Sau đây là một đối tượng hệ thống cùng các tham số và điều kiện đầu.

In [10]:
alpha = 7
dim = 3

t_0 = data.index[0]
h_0 = data[t_0]
t_end = data.index[-1]

system = System(alpha=alpha, 
                dim=dim, 
                h_0=h_0, 
                t_0=t_0, 
                t_end=t_end)

Và sau đây là một hàm cập nhật. Hàm này nhận vào tham số là chiều cao hiện thời và trả lại chiều cao tại bước thời gian kế tiếp.

In [11]:
def update(height, t, system):
    """Cập nhật chiều cao dựa theo mô hình cấp số nhân.
    
    height: chiều cao hiện thời tính bằng feet
    t: năm tính toán
    system: đối tượng hệ thống cùng với tham số mô hình
    """
    area = height**2
    mass = height**system.dim
    mass += system.alpha * area
    return mass**(1/system.dim)

Tôi sẽ thử hàm cập nhật với các điều kiện ban đầu.

In [12]:
update(h_0, t_0, system)

Đây là phiên bản `run_simulation` thông thường của ta.

In [13]:
def run_simulation(system, update_func):
    """Mộ phỏng hệ thống bằng một hàm cập nhật bất kì.
    
    system: đối tượng hệ thống System
    update_func: hàm cập nhật, để tính số cá thể trong năm tới
    
    trả lại: TimeSeries
    """
    results = TimeSeries()
    results[system.t_0] = system.h_0
    
    for t in linrange(system.t_0, system.t_end-1):
        results[t+1] = update_func(results[t], t, system)
        
    return results

Và sau đây là cách ta chạy nó.

In [14]:
results = run_simulation(system, update)
results.tail()

Và kết quả sẽ trông như sau:

In [15]:
def plot_results(results, data):
    results.plot(style=':', label='model', color='gray')
    data.plot(label='data')
    decorate(xlabel='Time (years)',
             ylabel='Height (feet)')
    
plot_results(results, data)

Mô hình đã hội tụ về một đường thẳng.

Tôi đã chọn giá trị của `alpha` sao cho càng khớp điểm số liệu bao nhiêu thì tốt bấy nhiêu, nhưng rõ ràng ở số liệu có độ cong mà mô hình không bắt được.

Đây là những sai số, tức là khác biệt giữa mô hình cây và dữ liệu.

In [16]:
errors = results - data
errors.dropna()

Và đây là sai số tuyệt đối trung bình.

In [17]:
def mean_abs_error(results, data):
    return (results-data).abs().mean()

mean_abs_error(results, data)

Mô hình này có thể giải thích tại sao chiều cao của cây tăng gần như là tuyến tính:

1. Nếu diện tích tỉ lệ với $h^2$ còn khối lượng thì tỉ lệ với $h^3$, và

2. Mức thay đổi về khối lượng thì tỉ lệ với diện tích, và

3. Chiều cao tăng trưởng một cách tuyến tính, thì khi đó:

4. Diện tích tăng tỉ lệ với $h^2$, và

5. Khối lượng tăng tỉ lệ với $h^3$.

Nếu mục đích là chỉ nhằm giải thích (xấp xỉ) sự tăng trưởng tuyến tính, thì ta có thể dừng lại ở đây. Song mô hình vẫn chưa khớp với dữ liệu lắm, và mô hình hàm ý rằng cây vẫn cứ tiếp tục tăng trưởng mãi.

Vì vậy ta cần phải làm tốt hơn nữa.

## Mô hình 2

Lần thử thứ hai, hãy coi như ta không biết $D$. Mà thực tế là ta không biết, vì cây cối không như các khối vật thể đơn giản; chúng giống các phân mảnh (fractal) hơn, chúng có [chiều fractal](https://en.wikipedia.org/wiki/Fractal_dimension).

Tôi dự trù rằng số chiều fractal của cây sẽ vào khoảng giữa 2 và 3, nên tôi đoán 2.5.

In [18]:
alpha = 7
dim = 2.5

Tôi sẽ bọc mã lệnh ở mục truwsowsc vào trong một hàm. Hàm này nhận các tham số đầu vào và tạo nên một đối tượng `System`.

In [19]:
def make_system(params, data):
    """Tạo một đối tượng System.
    
    params: một dãy các alpha, dim
    data: dãy Series
    
    trả lại: đối tượng System
    """
    alpha, dim = params
    
    t_0 = data.index[0]
    t_end = data.index[-1]
    h_0 = data[t_0]

    return System(alpha=alpha, dim=dim, 
                  h_0=h_0, t_0=t_0, t_end=t_end)

Dưới đây là cách dùng nó.

In [20]:
params = alpha, dim
system = make_system(params, data)

Với các giá trị tham số khác nhau, ta thu được những đường cong với ứng xử khác nhau. Tôi đã chọn thủ công được những đường sau đây.

In [21]:
def run_and_plot(alpha, dim, data):
    params = alpha, dim
    system = make_system(params, data)
    results = run_simulation(system, update)
    results.plot(style=':', color='gray', label='_nolegend')

In [22]:
run_and_plot(0.145, 2, data)
run_and_plot(0.58, 2.4, data)
run_and_plot(2.8, 2.8, data)
run_and_plot(6.6, 3, data)
run_and_plot(15.5, 3.2, data)
run_and_plot(38, 3.4, data)

data.plot(label='data')
decorate(xlabel='Time (years)',
             ylabel='Height (feet)')

Để tìm ra các tham số khớp với số liệu nhất, tôi sẽ dùng `leastsq`.

Ta cần một hàm sai số nhận vào các tham số rồi trả lại sai số:

In [23]:
def error_func(params, data, update_func):
    """Chạy mô hình và trả lại sai số.
    
    params: dãy gồm các alpha, dim
    data: dãy Series
    update_func: đối tượng hàm
    
    returns: dãy các sai số
    """
    print(params)
    system = make_system(params, data)
    results = run_simulation(system, update_func)
    return (results - data).dropna()

Dưới đây là cách ta sử dụng nó:

In [24]:
errors = error_func(params, data, update)

Bây giờ ta có thể truyền `error_func` vào `leastsq`, hàm thứ hai này tìm ra các tham số để giảm thiểu binh phương của các sai số.

In [25]:
best_params, details = leastsq(error_func, params, data, update)

In [26]:
print(details.success)

Dùng các tham số tốt nhất vừa tìm được, ta có thể chạy mô hình và vẽ biểu đồ kết quả.

In [27]:
system = make_system(best_params, data)
results = run_simulation(system, update)
plot_results(results, data)

Sai số tuyệt đối trung bình lần này thì tốt hơn của Mô hình 1, song điều đó không nhiều ý nghĩa. Mô hình vẫn chưa khớp với số liệu.

In [28]:
mean_abs_error(results, data)

Và số chiều fractal ước tính được bằng 3.11, vốn có vẻ không ổn.

Ta hãy thử thêm một lần nữa.

## Mô hình 3

Các mô hình 1 và 2 đều hàm ý rằng cây mọc cao mãi, nhưng ta biết điều đó không đúng.  that trees can grow forever, but we know that's not true. Khi cây càng cao thì càng khó để đưa nước và dưỡng chất lên dưới ảnh hưởng của trọng lực, và do vậy sự tăng trưởng cũng chậm lại.

Ta có thể mô hình hóa hiệu ứng này bằng cách thêm một hệ số vào mô hình tương tự như ta đã thấy ở mô hình logit về tăng trưởng dân số. Thay vì giả sử rằng:

$ m_{n+1} = m_n + \alpha A $ 

Hãy giả sử rằng

$ m_{n+1} = m_n + \alpha A (1 - h / K) $

trong đó $K$ tương tự như sức chứa trong mô hình logit. Khi $h$ tiến đến $K$, thừa số $(1 - h/K)$ sẽ tiến đến 0, dẫn đến mức tăng trưởng bị chậm đi.

Dưới đây là mã lệnh thực hiện mô hình với sự sửa đổi này:

In [30]:
alpha = 2.0
dim = 2.5
K = 150

params = [alpha, dim, K]

Đây là một phiên bản cập nhật của `make_system`

In [31]:
def make_system(params, data):
    """Tạo một đối tượng System.
    
    params: dãy các giá trị alpha, dim, K
    data: một dãy Series
    
    trả lại: đối tượng System
    """
    alpha, dim, K = params
    
    t_0 = data.index[0]
    t_end = data.index[-1]
    h_0 = data[t_0]

    return System(alpha=alpha, dim=dim, K=K, 
                  h_0=h_0, t_0=t_0, t_end=t_end)

Sau đây là đối tượng `System` mới.

In [32]:
system = make_system(params, data)

Và đây là hàm cập nhật mới.

In [33]:
def update3(height, t, system):
    """Cập nhật chiều cao dựa trên mô hình cấp số nhân với số hạng hạn chế tăng trưởng.
    
    height: chiều cao hiện thời, tính bằng feet
    t: năm hiện thời
    system: đối tượng hệ thống với các tham số mô hình
    """
    area = height**2
    mass = height**system.dim
    mass += system.alpha * area * (1 - height/system.K)
    return mass**(1/system.dim)

Như thường lệ, ta sẽ kiểm thử hàm cập nhật với các điều kiện ban đầu.

In [34]:
update3(h_0, t_0, system)

Và ta sẽ kiểm thử hàm sai số với hàm cập nhật mới viết.

In [35]:
error_func(params, data, update3)

Bây giờ hãy dò tìm các tham số phù hợp nhất.

In [36]:
best_params, details = leastsq(error_func, params, data, update3)

In [37]:
details.success

Với các tham số này, ta có thể khớp dữ liệu tốt hơn nhiều.

In [38]:
system = make_system(best_params, data)
results = run_simulation(system, update3)
plot_results(results, data)

Và sai số tuyệt đối trung bình sẽ nhỏ đi đáng kể.

In [39]:
mean_abs_error(results, data)

Số chiều fractal vừa ước tính bằng khoảng 2.6, vậy là hợp lý; nó ngụ ý rằng nếu bạn tăng gấp đôi chiều cao của cây thì khối lượng sẽ tăng lên $2^{2.6}$ lần.

In [40]:
2**2.6

Nói cách khác, khối lượng của cây tăng nhanh hơn diện tích mặt gỗ, nhưng không nhanh bằng thể tích khối gỗ 3 chiều.

Mô hình này có tác dụng gì?

1) Nó cho ta một lời giải thích khả dĩ cho hình dạng đường cong tăng trưởng của cây.

2) Nó cho ta một cách ước tính số chiều fractal của một cái cây dựa trên đường cong tăng trưởng (có thẻ là với những giá trị khác nhau tùy theo loài).

3) Nó có thể cho ta một cách dự tính mức tăng trưởng của một cái cây trong tương lại, dựa theo con số đo đạc trong quá khứ. Cũng như với mô hình dân số logit, điều này có lẽ chỉ đúng khi ta đã quan sát đến phần mà đường cong thể hiện tốc độ tăng trưởng giảm dần.

## Phân tích

Với sự giúp đỡ từ đồng nghiệp của tôi, John Geddes, chúng ta có thể phân tích một chút.

Bắt đầu từ phương trình sai phân cho khối lượng:
 
$m_{n+1} = m_n + \alpha A (1 - h / K) $

Ta có thể viết các phương trình vi phân tương ứng sau:

(1) $ \frac{dm}{dt} = \alpha A (1 - h / K) $

Với

(2) $A = h^2$

và

(3) $m = h^D$

Lấy đạo hàm của phương trình cuối, ta được

(4) $\frac{dm}{dt} = D h^{D-1} \frac{dh}{dt}$

Kết hợp (1), (2), và (4), ta có thể viết một phương trình vi phân cho $h$:

(5) $\frac{dh}{dt} = \frac{\alpha}{D} h^{3-D} (1 - h/K)$

Bây giờ hãy xét hai trường hợp:

* Với $K$ vô hạn, thừa số $(1 - h/K)$ tiến đến 1, và ta có Mô hình 2.

* Với $K$ hữu hạn, ta có Mô hình 3.

### Mô hình 2

Trong Mô hình 2, ta sẽ xét hai trường hợp đặc biệt, với $D=2$ và $D=3$.

Với $D=2$, ta có

$\frac{dh}{dt} = \frac{\alpha}{2} h$

và phương trình này cho ta tăng trưởng theo cấp số nhân với tham số $\alpha/2$.

Với $D=3$, ta có Mô hình 1, cùng phương trình:

$\frac{dh}{dt} = \frac{\alpha}{3}$

vốn cho ta tăng trưởng tuyến tính với tham số $\alpha/3$. Kết quả này giải thích tại sao Mô hình 1 có dạng tuyến tính.

### Mô hình 3

Trong Mô hình 3, ta sẽ xét hai trường hợp đặc biệt, với $D=2$ và $D=3$.

Với $D=2$, ta có

$\frac{dh}{dt} = \frac{\alpha}{2} h (1 - h/K)$

và phương trình này cho ta tăng trưởng logit với các tham số $r = \alpha/2$ và $K$.

Với $D=3$, ta có

$\frac{dh}{dt} = \frac{\alpha}{3} (1 - h/K)$

vốn cho ta tăng trưởng bậc nhất với phản hồi bậc thang; nghĩa là nó hội tụ về $K$ như một hàm mũ âm:

$ h(t) = c \exp(-\frac{\alpha}{3K} t) + K $

trong đó $c$ là một hằng số phụ thuộc vào điều kiện ban đầu.

**Bài tập mở** Hãy tìm một nghiệm giải tích khi $D$ nằm giữa 2 và 3, và so sánh nó với số liệu. Lưu ý: Tham số mà ta đã ước lượng cho phương trình sai phân có thể không đúng cho phương trình vi phân.

Tài liệu tham khảo:

Garcia, [A stochastic differential equation model for the
height growth of forest stands](http://citeseerx.ist.psu.edu/viewdoc/download;jsessionid=664FED1E46ABCBF6E16741C294B79976?doi=10.1.1.608.81&rep=rep1&type=pdf)

[Phần mềm EasySDE cùng số liệu](http://forestgrowth.unbc.ca/)